In [1]:
import os
import math
import pandas as pd 
from dotenv import load_dotenv
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
import uuid


In [2]:
load_dotenv('../.env')

SHOULDER_BREADTH_INTERCEPT=os.getenv('SHOULDER-BREADTH-INTERCEPT')
assert(SHOULDER_BREADTH_INTERCEPT)

SHOULDER_BREADTH_COEF=os.getenv('SHOULDER-BREADTH-COEF')
assert(SHOULDER_BREADTH_COEF)

CHEST_INTERCEPT=os.getenv('CHEST-INTERCEPT')
assert(CHEST_INTERCEPT)

CHEST_COEF=os.getenv('CHEST-COEF')
assert(CHEST_COEF)

WAIST_INTERCEPT=os.getenv('WAIST-INTERCEPT')
assert(WAIST_INTERCEPT)

WAIST_COEF=os.getenv('WAIST-COEF')
assert(WAIST_COEF)

HIP_INTERCEPT=os.getenv('HIP-INTERCEPT')
assert(HIP_INTERCEPT)

HIP_COEF=os.getenv('HIP-COEF')
assert(HIP_COEF)

In [3]:
def find_border_non_negative_index(array, startingFromEnd):
    if startingFromEnd:
        for i in range(len(array) - 1, -1, -1):
            if array[i] != -1:
                return i
        return -1
    else:
        for i in range(len(array)):
            if array[i] != -1:
                # mirrr values in case of left side
                # Make the middle of the body (end of the left side) 0-> index 358 is widht-358-> X relative to middle of body
                return len(array)-i
        return -1

def get_white_contour_of_half_bodies(image_path):
    binary_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 2: Split the image into left and right halves
    height, width = binary_image.shape
    left_half = binary_image[:, :width // 2]
    right_half = binary_image[:, width // 2:]

    return left_half,right_half


def find_contour_pixels(binary_half,isRightSide):
    contour_pixels = []
    for row in range(binary_half.shape[0]):
        white_pixels = np.where(binary_half[row] == 255,binary_half[row],-1)

        if white_pixels.size > 0:
            last_white_pixel = find_border_non_negative_index(white_pixels, startingFromEnd = isRightSide)
            contour_pixels.append(last_white_pixel)
    return contour_pixels


def visualize_white_contour(half_body):
    x_values = list(range(len(half_body)))
    
    # Plot the array
    plt.figure(figsize=(10,aspect_ratio * 10))  # Adjust the multiplier as needed
    plt.plot(x_values, half_body, marker='o', linestyle='-', color='b')
    
    # Add labels and title
    plt.xlabel('Image')
    plt.ylabel('White pixels (contour)')
    plt.title('Slope of white contours of the body')
    
    # Show the plot
    plt.grid(True)
    plt.show()

def find_shoulder_coordinate(contour_white_pixel,half_body, isRightSide, window_size=15, isDebug=False):
    # Calculate the differences between consecutive elements within the window
    differences = np.diff(contour_white_pixel)
    gradual_jump_indices = []

    for i in range(len(differences) - window_size):
        if all(differences[i + j] > 0 for j in range(1, window_size + 1)):
            gradual_jump_indices.append(i + window_size)

    x_keypoint = 0
    y_keypoint = 0
    
    if(isRightSide):
        x_keypoint = contour_white_pixel[gradual_jump_indices[-1]]
        y_keypoint = gradual_jump_indices[-1]
    else:
        x_keypoint = half_body.shape[1]-contour_white_pixel[gradual_jump_indices[-1]]
        y_keypoint = gradual_jump_indices[-1]
    
    if(isDebug):
        fig, ax = plt.subplots(1)
        ax.imshow(half_body)
        ax.plot(x_keypoint, y_keypoint, 'bo')
        plt_name = str(uuid.uuid4()) + '.png'
        plt.savefig(f'./debug2/{plt_name}')
        plt.close()

    return x_keypoint,y_keypoint

In [4]:
def get_measurement_info(measurement_id, picture_name):
    """
    Helper function retrieves the requested measurement in cm from the metadata csv files
    
    :param measurement_id: Id of the measurement point
    :param picture_name: Name of the image
    """

    # First retrieve subject id based on photo name
    file = open("../Annotation/subject_to_photo_map.csv", 'r')
    subject_id = "NONE"
    for row in csv.reader(file):
        if row[1] == picture_name.split(".")[0]:
            subject_id = row[0]
            break

    # get measurement info for specific subject id
    file = open("../Annotation/measurements.csv", 'r')
    reader = csv.DictReader(file)
    for row in reader:
        if row["subject_id"] == subject_id:
            return row[measurement_id]

In [5]:
dataset_dir = '../Training/Regression/dataset2'
file_list = os.listdir(dataset_dir)

X_values = []
Y_values = []

for img in file_list:
    print(img)
    try:
        image_path = f'{dataset_dir}/{img}'
        left_half, right_half = get_white_contour_of_half_bodies(image_path)
        
        contour_white_pixel_right = find_contour_pixels(right_half, isRightSide=True)
        contour_white_pixel_left = find_contour_pixels(left_half, isRightSide=False)
            
        relative_x_right, relative_y_right = find_shoulder_coordinate(contour_white_pixel_right[:len(contour_white_pixel_right)//3],half_body=right_half,isRightSide=True,isDebug=True)
        relative_x_left, relative_y_left = find_shoulder_coordinate(contour_white_pixel_left[:len(contour_white_pixel_left)//3],half_body=left_half,isRightSide=False,isDebug=True)
    
        pixel_distance = math.sqrt((left_half.shape[1]+relative_x_right-relative_x_left)**2+(relative_y_right-relative_y_left)**2)
        print(pixel_distance)
        X_values.append(pixel_distance)
    
        measurement_cm = get_measurement_info('shoulder-breadth',img)
        Y_values.append(measurement_cm)
    except:
        print('##### ERROR #####')
        continue


df = pd.DataFrame({'Keypoint Pixel Distance':X_values, 'CM Distance':Y_values}) 
df.to_csv('tmp2.csv', index=False) 

4fbbb13717bcdf566caa2c87965c46a0.png
154.01298646542764
4d7cbe1e63ae2171e78ba826361f6400.png
155.0
5dccc1eea15308d659a05dafa950a08c.png
161.4465855941215
4ee727174d39826fc44bff4d9a2753fd.png
192.0
0c60a867eac27ae952fe5fe20ca0b777.png
176.0454486773231
3f90b5c290595c75c8b2d8b12132d2ed.png
155.11608556174951
6a945ceaf4d58c1f8cb2a7591eac08f8.png
##### ERROR #####
6f6a7be050995ce1eab47a2a5b71d1f2.png
142.0
6cdb7ec5358452e20ca138d46336fc8f.png
167.0029939851379
01ec2085a12d883610f6bddf662d43aa.png
173.0115603073968
6f5dc9ad29b971d15a0022e2e7f5ba1d.png
##### ERROR #####
1c5820868879bea0cdb98a1a3fc6dd3f.png
164.1097193952875
2c0b52eaec1c06af8e1d845bad1d4496.png
147.0
3e692622001997822021f89ea6e21c7f.png
219.18485349129398
6a4bb4d52482dd52b33c7bcaac267ca6.png
183.02458851203573
0bc1c97be6cfb2ac29639c1c3be1ec60.png
151.0529708413575
1bc758d5f99ebfdbbaf9b1a31b9a1cec.png
166.24379687675568
6d10659246323e375d3e14fc529fe906.png
147.0
3a45528a3a6c986d716210c5ee567545.png
162.01234520862909
4afa3f5dc